# The Importance of Cleaning the Text

After a few different iterations, I think that I have found a pretty good way to clean the questions to improve the performance of a model. I was able to reduce my loss value by a few points because of this method.  Feel free to use this code and improve upon the method!

In [44]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
import pickle

In [26]:
data_path = "../../"
train = pd.read_csv(data_path + "train.csv")[10000:20000]
test = pd.read_csv(data_path + "test.csv")[10000:20000]

In [27]:
train = train.reset_index()
test = test.reset_index()

In [28]:
train.shape

(10000, 7)

In [29]:
# Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

index           0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64
index        0
test_id      0
question1    0
question2    0
dtype: int64


In [30]:
# Add the string 'empty' to empty strings
train = train.fillna('empty')
test = test.fillna('empty')

In [31]:
# Preview some of the pairs of questions
a = 0 
for i in range(a,a+10):
    print(train.question1[i])
    print(train.question2[i])
    print()

Is it safe for a woman to travel alone in Vietnam?
Is it safe for a woman to travel alone in Japan?
()
What do paving stones cost?
What are "paving stones"?
()
Is it an irony of Bengaluru that the people who come from other states are its biggest critics?
Have you read 50 Shades Of Grey? If so, did it turn you on?
()
What are good pop songs to sing in bass/baritone range?
What are some good up-tempo musical theater audition songs for baritone/tenor?
()
What is the best free antivirus for a PC in 2015?
What are the best free antivirus programs for PC?
()
What dating apps do Indians who are living in the US use?
Can an Indian guy date attractive white girls in the US if he isn't rich?
()
What is the best test application?
Do I get views for a anonymous question?
()
Why isn't Hillary in prison?
Should Hillary Clinton be in prison?
()
When will the online registration and choice filling for WBJEE medical 2016 start?
Should Maharashtra become an independent country seeing its GDP figures? I

In [32]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

In [33]:
def text_to_wordlist(text, remove_stop_words=False, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "what", text)
    text = re.sub(r"What's", "What", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [34]:
def process_questions(question_list, questions, question_list_name, dataframe):
    '''transform questions and display progress'''
    for question in questions:
        question_list.append(text_to_wordlist(question))
        if len(question_list) % 100000 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [35]:
train_question1 = []
process_questions(train_question1, train.question1, 'train_question1', train)

In [36]:
train_question2 = []
process_questions(train_question2, train.question2, 'train_question2', train)

In [37]:
train.loc[:,"question1"] = train_question1
train.loc[:,"question2"] = train_question2

In [38]:
train.to_csv("../../dev_clean.csv")

In [39]:
test_question1 = []
process_questions(test_question1, test.question1, 'test_question1', test)

In [40]:
test_question2 = []
process_questions(test_question2, test.question2, 'test_question2', test)

In [41]:
# Preview some transformed pairs of questions
a = 0 
for i in range(a,a+10):
    print(train_question1[i])
    print(train_question2[i])
    print()

Is it safe for a woman to travel alone in Vietnam 
Is it safe for a woman to travel alone in Japan 
()
What do paving stones cost 
What are paving stones 
()
Is it an irony of Bengaluru that the people who come from other states are its biggest critics 
Have you read 50 Shades Of Grey If so did it turn you on 
()
What are good pop songs to sing in bass baritone range 
What are some good up tempo musical theater audition songs for baritone tenor 
()
What is the best free antivirus for a PC in 2015 
What are the best free antivirus programs for PC 
()
What dating apps do Indians who are living in America use 
Can an Indian guy date attractive white girls in America if he isn t rich 
()
What is the best test application 
Do I get views for a anonymous question 
()
Why isn t Hillary in prison 
Should Hillary Clinton be in prison 
()
When will the online registration and choice filling for WBJEE medical 2016 start 
Should Maharashtra become an independent country seeing its GDP figures Is i

In [42]:
len(train_question1)

10000

In [45]:
pickle.dump([list(train_question1) + list(train_question2)], open("../corpus/sentences_dev_10000.pkl", "w"))